<a href="https://colab.research.google.com/github/urjad/Coding-Assingments/blob/main/ANN_with_callbacks_Tensorboard%2C_Early_Stopping%2C_Model_Checkpointing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
ROOT="/content/drive/MyDrive/Colab Notebooks"
os.chdir(ROOT)

In [ ]:
os.listdir()

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
tf.keras.__version__

In [ ]:
tf.config.list_physical_devices("GPU")

In [ ]:
tf.config.list_physical_devices("CPU")

### A Simple classifier using keras's Sequential API

Dataset MNIST


1.   Its a dataset of handwritten digits
2.   each data point is 2d array of 28x28.
3. It has 10 classes 0-10



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os

In [ ]:
mnist= tf.keras.datasets.mnist #Modified National Institute of Standards and Technology dataset

In [ ]:
(X_train_full,y_train_full),(X_test,y_test)=mnist.load_data()

In [ ]:
X_train_full.shape,y_train_full.dtype

In [ ]:
y_test.dtype

In [ ]:
X_test.shape,X_test.dtype

In [ ]:
X_valid,X_train=X_train_full[:5000]/255,X_train_full[5000:]/255
y_valid,y_train=y_train_full[:5000],y_train_full[5000:]

X_test=X_test/255

In [ ]:
plt.imshow(X_train[0],cmap="binary")
plt.axis("off")
plt.show()
y_train[0]

Writing som sample to Tensorboard


In [ ]:
import time

def get_timestamp():
  return time.asctime().replace(" ","_").replace(":","")

In [ ]:
get_timestamp()

In [ ]:
def get_log_dir_path(log_dir="logs/fit"):
  log_path = os.path.join(log_dir,get_timestamp())
  print(f"logs will be saved at:{log_path}")
  return log_path

log_dir=get_log_dir_path()
log_dir

In [ ]:
file_writer = tf.summary.create_file_writer(logdir=log_dir)

In [ ]:
with file_writer.as_default():
  images = np.reshape(X_train[10: 30], (-1, 28, 28, 1))
  tf.summary.image("20 handwritten digit samples", images, max_outputs=25, step=0)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
plt.imshow# library is used to display data as an image; i.e. on a 2D regular raster.

In [ ]:
plt.imshow(X_train[3])
#plt.show()
y_train[3]

In [ ]:
import seaborn as sns
plt.figure(figsize=(30,30))
sns.heatmap(X_train[3],annot=True,cmap="binary")

In [ ]:
LAYERS = [
          tf.keras.layers.Flatten(input_shape=[28,28], name="inputLayer"),
          tf.keras.layers.Dense(300,activation="relu", name="hiddenLayer1"),
          tf.keras.layers.Dense(100,activation="relu", name="hiddenLayer2"),
          tf.keras.layers.Dense(10,activation="softmax", name="outputLayer"),
]


In [ ]:
model=tf.keras.models.Sequential(LAYERS)

In [ ]:
model.layers

In [ ]:
model.summary()

In [ ]:
hidden1=model.layers[1]
model.get_layer(hidden1.name)

In [ ]:
hidden1 = model.layers[1]

model.get_layer(hidden1.name)

In [ ]:
hidden1.get_weights()

In [ ]:
weights,bias=hidden1.get_weights()

In [ ]:
weights

In [ ]:
bias

In [ ]:
weights.shape

In [ ]:
bias.shape

In [ ]:
LOSS_FUNCTION="sparse_categorical_crossentropy"
OPTIMIZER="SGD"
METRICS= ['accuracy']

model.compile(loss=LOSS_FUNCTION,
              optimizer=OPTIMIZER,
              metrics=METRICS)

### Important -

* **Epochs**: Integer. 
    
    Number of epochs to train the model. An epoch is an iteration over the entire x and y data provided. 

* **batch_size**: Integer or None. 
    
    Number of samples per gradient update. If unspecified, batch_size will default to 32. 
    
    *NOTE:* Do not specify the batch_size if your data is in the form of datasets, generators, or keras.utils.Sequence instances (since they generate batches).

* **validation_batch_size**: Integer or None. 

    Number of samples per validation batch. 
    
    If unspecified, will default to batch_size. 
    
    *NOTE:* Do not specify the validation_batch_size if your data is in the form of datasets, generators, or keras.utils.Sequence instances (since they generate batches).

[Reference](https://www.tensorflow.org/api_docs/python/tf/keras/Model?hl=zh-tw#fit)

[Another Referencce](https://keras.io/api/)

In [ ]:
EPOCHS = 30
VALIDATION_SET=(X_valid,y_valid)

history = model.fit(X_train,y_train,
                    epochs=EPOCHS,
                    validation_data=VALIDATION_SET)

In [ ]:
55000 / 16


In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
model.save("model.h5")

In [ ]:
loaded_model=tf.keras.models.load_model("model.h5")

In [ ]:
loaded_model.evaluate(X_test,y_test)

In [ ]:
pd.DataFrame(history.history)

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1) #gca? set_ylim?
plt.show()

In [ ]:
X_new = X_test[:3]
y_new= y_test[:3]
y_proba=model.predict(X_new)
y_proba.round(2)

In [ ]:
y_pred=np.argmax(y_proba,axis=-1)
y_pred,y_new

In [ ]:
for data,pred,actual in zip(X_new,y_pred,y_new):
  plt.imshow(data,cmap="binary")
  plt.title(f"Predicted: {pred}, Actual: {actual}")
  plt.axis("off")
  plt.show()
  print("---"*20)